In [ ]:
# ecoera
# O:C ratio is 0.039
# 78% carbon content per ton of biochar
# so 0.78 tons of C or 2.86 tons of CO2e per ton of biochar
# they are selling 5000 tons of CO2e
# which is 5000/2.86 tons of biochar
# (puro additionally discounts that by 0.025)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
from numpy import arange, zeros, exp, log

In [ ]:
years = arange(0,1000)
def sim(C1, C2, k1, j2):
    y1 = zeros(years.shape)
    y2 = zeros(years.shape)
    for t in years:
        if t == 0:
            y1[t] = C1
            y2[t] = C2
        else:
            y1[t] = y1[t-1] * exp(-k1)
            y2[t] = y2[t-1] * exp(-k2)
    return y1, y2

### Ecoera

In [ ]:
volume_CO2 = 5000
volume_C = volume_CO2 / 3.67
volume_C_biochar = volume_C / 0.78
O_C_ratio = 0.039

In [ ]:
volume_C_biochar

In [ ]:
f = 0.6
k1 = 0.03
C1 = volume_C_biochar / f
k2 = log(2)/2000
C2 = volume_C_biochar

In [ ]:
y1, y2 = sim(C1, C2, k1, k2)

In [ ]:
plt.plot(years,y1)
plt.plot(years,y2-y1);

### Carbofex

In [ ]:
volume_CO2 = 2500
volume_C = volume_CO2 / 3.67
volume_C_biochar = volume_C / 0.92
O_C_ratio = 0.021
volume_C

In [ ]:
f = 0.6
k1 = 0.03
C1 = volume_C_biochar / f
k2 = log(2)/1000
C2 = volume_C_biochar

In [ ]:
y1, y2 = sim(C1, C2, k1, k2)

In [ ]:
plt.plot(years,y2-y1)
plt.plot(years,y2);

### CarboCulture

In [ ]:
3.12/3.67

In [ ]:
volume_CO2 = 62
volume_C = volume_CO2 / 3.67
volume_C_biochar = volume_C / 0.85
O_C_ratio = 0.08
volume_C

In [ ]:
f = 0.6
k1 = 0.03
C1 = volume_C_biochar / f
k2 = log(2)/1000
C2 = volume_C_biochar

In [ ]:
k2

In [ ]:
y1, y2 = sim(C1, C2, k1, k2)

In [ ]:
plt.plot(years,y2-y1)